# Data

> Setup the data

In [ ]:
#| default_exp data

In [ ]:
#| export
import numpy as np
import yaml
import os

Carregando os dicionários de mapeamento do .yaml do semantic kitti

In [ ]:
#| export
folder_path = "../point_clouds/semantic_kitti/"

with open(folder_path+"semantic-kitti.yaml", 'r') as file:
    data = yaml.safe_load(file)

label_map = data.get('labels', {})
learning_map = data.get('learning_map', {})
learning_map_inv = data.get('learning_map_inv', {})
color_map = data.get('color_map', {})

Carrega a point cloud com base em um range.
* Args:
  * velodyne_path: caminho da pasta que contém os .bins
  * velodyne_list: lista contendo os bins que devem ser carregados, parâmetro definido em 02_image_generator.ipynb
  * labels_path: caminho da pasta que contem os .labels 
  * label_list: lista contendo os labels que devem ser carregados, parâmetro definido em 02_image_generator.ipynb
* Return:
  * point_cloud: np array contendo a núvem de pontos, onde cada dimensão corresponde a uma feature diferente

In [ ]:
#| export
def load_data(velodyne_path, velodyne_list, labels_path=None, label_list=None):
    point_cloud = []
    for index in (range(len(velodyne_list))):
        frame_path = os.path.join(velodyne_path, velodyne_list[index])
        with open(frame_path, 'rb') as f:
            frame = np.fromfile(f, dtype=np.float32).reshape(-1, 4)

        if labels_path != None:
            label_path = os.path.join(labels_path, label_list[index])
            with open(label_path, 'rb') as f:
                label = np.fromfile(f, dtype=np.uint32)
                label = label & 0xFFFF
            point_cloud.append(np.concatenate((frame, label.reshape(-1, 1)), axis=1))
        else:
            point_cloud.append(frame)
            
    return point_cloud

Carrega a point cloud por completo (utilizada somente para testes).

In [ ]:
#| export
def load_all_data(sequence_path):
    velodyne_path = os.path.join(sequence_path, 'velodyne')
    labels_path = os.path.join(sequence_path, 'labels')

    frames = []
    for frame_name in sorted(os.listdir(velodyne_path)):
        frame_path = os.path.join(velodyne_path, frame_name)
        with open(frame_path, 'rb') as f:
            frames.append(np.fromfile(f, dtype=np.float32).reshape(-1, 4))

    labels = []
    for label_name in sorted(os.listdir(labels_path)):
        label_path = os.path.join(labels_path, label_name)
        with open(label_path, 'rb') as f:
            np_label = np.fromfile(f, dtype=np.uint32)
            np_label = np_label & 0xFFFF
            labels.append(np_label)

    point_cloud = []
    for frame, label in zip(frames,labels):
        point_cloud.append(np.concatenate((frame, label.reshape(-1, 1)), axis=1))
    
    return point_cloud

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()